# Uzun Kısa Süreli Bellek (Long Short Term Memory) (LSTM)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter # Kelime frekanslarını hesaplama
from itertools import product # Grid Search için kombinasyon oluşturma

**LSTM** ile metin üretme . Bellek sorunlarını çözmek için geliştirilmiş bir tür yinelemeli sinir ağı .

In [2]:
# Ürün Yorumları
text = """Bu ürün beklentimi fazlasıyla karşıladı.  
Malzeme kalitesi gerçekten çok iyi.  
Kargo hızlı ve sorunsuz bir şekilde elime ulaştı.  
Fiyatına göre performansı harika.  
Kesinlikle tavsiye ederim ve öneririm!"""

In [3]:
# Veri Ön İşleme
# Noktalama İşaretlerinden Kurtul
# Küçük Harf Dönüşümü
# Kelimeleri böl
words = text.replace(".","").replace("!","").lower().split()

In [4]:
# Kelime frekanslarını hesaplama 
word_counts = Counter(words)
vocab = sorted(word_counts , key = word_counts.get , reverse = True) # Kelimelerin frekansı
# Büyükten küçüğe sıralama 
word_to_ix = {word : i for i , word in enumerate(vocab)}
ix_to_word = {i : word for i , word in enumerate(vocab)}

# Eğitim verisi hazırlama 
data = [(words[i] , words[i + 1] ) for i in range(len(words) - 1)]

In [5]:
data

[('bu', 'ürün'),
 ('ürün', 'beklentimi'),
 ('beklentimi', 'fazlasıyla'),
 ('fazlasıyla', 'karşıladı'),
 ('karşıladı', 'malzeme'),
 ('malzeme', 'kalitesi'),
 ('kalitesi', 'gerçekten'),
 ('gerçekten', 'çok'),
 ('çok', 'iyi'),
 ('iyi', 'kargo'),
 ('kargo', 'hızlı'),
 ('hızlı', 've'),
 ('ve', 'sorunsuz'),
 ('sorunsuz', 'bir'),
 ('bir', 'şekilde'),
 ('şekilde', 'elime'),
 ('elime', 'ulaştı'),
 ('ulaştı', 'fiyatına'),
 ('fiyatına', 'göre'),
 ('göre', 'performansı'),
 ('performansı', 'harika'),
 ('harika', 'kesinlikle'),
 ('kesinlikle', 'tavsiye'),
 ('tavsiye', 'ederim'),
 ('ederim', 've'),
 ('ve', 'öneririm')]

**LSTM Modelin Oluşturulması**

In [6]:
class LSTM(nn.Module):
    
    def __init__(self , vocab_size , embedding_dim , hidden_dim):
        super(LSTM,self).__init__() # Bir üst sınıfın constructor ini çağırma 
        self.embedding = nn.Embedding(vocab_size , embedding_dim)
        self.lstm = nn.LSTM(embedding_dim , hidden_dim) # LSTM Katmanı
        self.fc = nn.Linear(hidden_dim , vocab_size)

    def forward(self , x):
        """
            input -> embedding -> lstm -> fc -> output
        """
        
        x = self.embedding(x) 
        lstm_out , _ = self.lstm(x.view(1,1,-1))
        output = self.fc(lstm_out.view(1,-1))
        return output

In [7]:
model = LSTM(len(vocab) , embedding_dim = 8 , hidden_dim = 32)

**Hiper Parametrelerin Belirlenmesi**

In [9]:
# Kelime Listesi -> Tensor
def prepare_sequence(seq , to_ix):
    return torch.tensor([to_ix[w] for w in seq] , dtype = torch.long )

# Hyperparameter tuning kombinasyonunun belirlenmesi
embedding_size = [8 , 16] # Denenecek embedding boyutları
hidden_sizes = [32 , 64] # Denenecek gizli kataman boyutları
learning_rates = [0.01 , 0.005] # Öğrenme oranı

best_loss = float("inf") # En düşük kayıp değerini saklamak için bir değişken
best_params = {} # En iyi parametreleri saklamak için boş bir dictionary


**Grid Search**

In [13]:
for emb_size , hidden_size , lr in product(embedding_size , hidden_sizes , learning_rates):
    print(f"Embedding :{emb_size}  -  Hidden Size : {hidden_size} - Learning Rate : {lr} ")

    # Modeli Tanımla
    model = LSTM(len(vocab) , emb_size , hidden_size) # Seçilen parametreler ile model oluşturma
    loss_function = nn.CrossEntropyLoss() # Entropi kayıp fonksiyonu
    optimizer = optim.Adam(model.parameters() , lr = lr) # Seçilen lr ile Adam optimizeri

    epochs = 50
    total_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0 
        
        for word , next_word in data :
        
            model.zero_grad() # Gradyanları Sıfırla
            input_tensor = prepare_sequence([word] , word_to_ix) # Girdiyi tensore çevir
            target_tensor = prepare_sequence([next_word] , word_to_ix) # Hedef kelimeyi tensore çevirme
            output = model(input_tensor ) # Prediction
            loss = loss_function(output , target_tensor)
            loss.backward() # Geri yayılım işlemi uygulama
            optimizer.step() # Parametreleri güncelle
            epoch_loss += loss.item()

        if epoch % 10 == 0:
            print(f"Epochs : {epoch} , Loss : {epoch_loss:.5f}")
        total_loss = epoch_loss

    # En İyi Modeli Kaydedelim
    if total_loss < best_loss:
        best_loss = total_loss
        best_params = {"embedding_dim":emb_size , "hidden_dim":hidden_size , "learning_rate":lr}
    print()
print(f"Best Params : {best_params}")

Embedding :8  -  Hidden Size : 32 - Learning Rate : 0.01 
Epochs : 0 , Loss : 86.89599
Epochs : 10 , Loss : 3.85817
Epochs : 20 , Loss : 2.29824
Epochs : 30 , Loss : 2.01193
Epochs : 40 , Loss : 1.89713

Embedding :8  -  Hidden Size : 32 - Learning Rate : 0.005 
Epochs : 0 , Loss : 85.34375
Epochs : 10 , Loss : 14.47495
Epochs : 20 , Loss : 3.61015
Epochs : 30 , Loss : 2.45563
Epochs : 40 , Loss : 2.08991

Embedding :8  -  Hidden Size : 64 - Learning Rate : 0.01 
Epochs : 0 , Loss : 86.60152
Epochs : 10 , Loss : 2.63809
Epochs : 20 , Loss : 2.07843
Epochs : 30 , Loss : 1.93907
Epochs : 40 , Loss : 1.85579

Embedding :8  -  Hidden Size : 64 - Learning Rate : 0.005 
Epochs : 0 , Loss : 85.76938
Epochs : 10 , Loss : 4.82903
Epochs : 20 , Loss : 2.33944
Epochs : 30 , Loss : 2.00226
Epochs : 40 , Loss : 1.86772

Embedding :16  -  Hidden Size : 32 - Learning Rate : 0.01 
Epochs : 0 , Loss : 86.42317
Epochs : 10 , Loss : 3.03618
Epochs : 20 , Loss : 2.14498
Epochs : 30 , Loss : 1.95486
Epochs

**En iyi parametreler ile eğitim**

In [14]:
final_model = LSTM(len(vocab),best_params["embedding_dim"] , best_params["hidden_dim"])
optimizer = optim.Adam(final_model.parameters(),lr = best_params["learning_rate"])
loss_function = nn.CrossEntropyLoss() # Entropi kayıp fonk.

epochs = 100
for epoch in range(epochs):
    epoch_loss = 0

    for word , next_word in data:
        final_model.zero_grad()
        input_tensor = prepare_sequence([word],word_to_ix)
        target_tensor = prepare_sequence([next_word],word_to_ix)
        output = final_model(input_tensor)
        loss = loss_function(output , target_tensor)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if epoch % 10 == 0 :
        print(f"Final Model Epoch : {epoch} , Loss : {epoch_loss:.5f}")

Final Model Epoch : 0 , Loss : 85.52355
Final Model Epoch : 10 , Loss : 3.26544
Final Model Epoch : 20 , Loss : 2.11454
Final Model Epoch : 30 , Loss : 1.89104
Final Model Epoch : 40 , Loss : 1.79326
Final Model Epoch : 50 , Loss : 1.73621
Final Model Epoch : 60 , Loss : 1.69809
Final Model Epoch : 70 , Loss : 1.67261
Final Model Epoch : 80 , Loss : 1.65028
Final Model Epoch : 90 , Loss : 1.63324


**Modelin Test Edilmesi**

In [16]:
# Kelime tahmin fonc
def predict_sequence(start_word , num_words):
    current_word = start_word # Şu anki kelime başlangıç kelimesi olarak ayarlanır
    output_sequence = [current_word] # Çıktı dizisi 

    for _ in range(num_words): # Belirlenen sayıda kelime tahmini
        with torch.no_grad():
            input_tensor = prepare_sequence([current_word] , word_to_ix) # kelime -> tensor
            output = final_model(input_tensor)
            predicted_ix = torch.argmax(output).item() # En yüksek olasılığa ait kelime indexi
            predicted_word = ix_to_word[predicted_ix] # İndexe karşılık gelen kelimeyi return et
            output_sequence.append(predicted_word)
            current_word = predicted_word # Bir sonraki tahmin için mevcut kelimeleri güncelle  
    return output_sequence 

In [17]:
start_word = "ürün"
num_predictions = 4
predicted_sequence = predict_sequence(start_word , num_predictions)
print(" ".join(predicted_sequence))

ürün beklentimi fazlasıyla karşıladı malzeme
